In [1]:
import os
import sys
sys.path.append('/zfsauton3/home/bparia/hf_moe/')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
from tabulate import tabulate

In [14]:
with open('../data/m5/data.pkl', 'rb') as fin:
    tree, ts_data, \
        (global_cont_feats, global_cat_feats, global_cat_dims) \
            = pickle.load(fin)

leaf_mat = tree.leaf_matrix
num_leaf = np.sum(leaf_mat, axis=1, keepdims=True)
cmat = leaf_mat / num_leaf

In [12]:
def get_cerror(path):
    levels = tree.levels
    with open(path, 'rb') as fin:
        pred, _ = pickle.load(fin)
    
    cpred = pred @ cmat.T

    errs = []
    for d in levels:
        sub_pred = pred[:, levels[d]]
        sub_cpred = cpred[:, levels[d]]
        diff = np.mean(np.abs(sub_cpred - sub_pred)) / np.mean(np.abs(sub_cpred))
        errs.append(diff)
    
    return errs

def get_average(model):
    errs = []
    for i in range(1, 11):
        path = os.path.join('../logs/', model, f'run_{i}', 'test.pkl')
        err = get_cerror(path)
        errs.append(err)
    
    errs = np.asarray(errs)
    errs = np.mean(errs, axis=0)
    print(errs)

In [5]:
get_average('favorita/hid_24_emb_8_timear/')
get_average('favorita/hid_24_emb_8_timear_noreg/')
get_average('favorita/hid_64_exp_1_norm/')

[0.00486112 0.00468465 0.00376906 0.        ]
[0.01218486 0.01349842 0.0101119  0.        ]
[0.04305472 0.04473148 0.04209769 0.        ]


In [11]:
get_average('tourism/hid_14_emb_6_timear/')
get_average('tourism/hid_14_emb_6_timear_noreg/')
get_average('tourism/hid_30_exp_1_norm/')


[0.09205788 0.07953036 0.06617345 0.06046085 0.        ]
[0.08513087 0.08244395 0.06709635 0.05924147 0.        ]
[0.09731197 0.0894319  0.08230363 0.08307251 0.        ]


In [15]:
get_average('m5/hid_42_emb_8_timear/')
get_average('m5/hid_42_emb_8_timear_noreg/')
get_average('m5/hid_64_exp_1_norm/')

[0.03165883 0.03390127 0.03432445 0.        ]
[0.03486171 0.03793999 0.03908627 0.        ]
[0.05221183 0.08916584 0.06242532 0.        ]
